# A simple blending problem

https://coin-or.github.io/pulp/CaseStudies/a_blending_problem.html

In [1]:
import pandas as pd 
import pulp

In [2]:
product = pd.read_csv('product_content.csv').set_index('Stuff')
product

,Protein,Fat,Fibre,Salt,Cost
Stuff,,,,,
Chicken,0.10,0.08,0.001,0.002,0.013
Beef,0.20,0.10,0.005,0.005,0.008
Mutton,0.15,0.11,0.003,0.007,0.010
Rice,0.00,0.01,0.100,0.002,0.002
Wheat,0.04,0.01,0.150,0.008,0.005
Gel,0.00,0.00,0.000,0.000,0.001


In [3]:
"""Creating problem obj"""
Prob = pulp.LpProblem("FoodProblem", pulp.LpMinimize)

In [4]:
"""Defining variables"""

variables = {
    name:pulp.LpVariable(
        name, lowBound=0, upBound=100, cat='Continuous'
    ) for name in product.index
}
variables

{'Chicken': Chicken,
 'Beef': Beef,
 'Mutton': Mutton,
 'Rice': Rice,
 'Wheat': Wheat,
 'Gel': Gel}

In [5]:
"""The objective is to minimize cost"""
Prob += pulp.lpSum(
    [product.loc[name, 'Cost']*v for name,v in variables.items()]
), "Objective"

In [6]:
"""Add constraints
sum(x_i) == 100
sum(protein*x_i) >= 8
"""

Prob += pulp.lpSum([v for v in variables.values()]) == 100, 'Percentage'
Prob += pulp.lpSum([product.loc[n, 'Protein']*v for n,v in variables.items()]) >= 8, 'Protein'
Prob += pulp.lpSum([product.loc[n, 'Fat']*v for n,v in variables.items()]) >= 6, 'Fat'
Prob += pulp.lpSum([product.loc[n, 'Fibre']*v for n,v in variables.items()]) <= 2, 'Fibre'
Prob += pulp.lpSum([product.loc[n, 'Salt']*v for n,v in variables.items()]) <= 0.4, 'Salt'

In [7]:
Prob.constraints

OrderedDict([('Percentage',
              1*Beef + 1*Chicken + 1*Gel + 1*Mutton + 1*Rice + 1*Wheat + -100 = 0),
             ('Protein',
              0.2*Beef + 0.1*Chicken + 0.15*Mutton + 0.04*Wheat + -8.0 >= 0),
             ('Fat',
              0.1*Beef + 0.08*Chicken + 0.11*Mutton + 0.01*Rice + 0.01*Wheat + -6.0 >= 0),
             ('Fibre',
              0.005*Beef + 0.001*Chicken + 0.003*Mutton + 0.1*Rice + 0.15*Wheat + -2.0 <= 0),
             ('Salt',
              0.005*Beef + 0.002*Chicken + 0.007*Mutton + 0.002*Rice + 0.008*Wheat + -0.4 <= 0)])

In [8]:
pulp.listSolvers()

['GLPK_CMD',
 'PYGLPK',
 'CPLEX_CMD',
 'CPLEX_PY',
 'GUROBI',
 'GUROBI_CMD',
 'MOSEK',
 'XPRESS',
 'PULP_CBC_CMD',
 'COIN_CMD',
 'COINMP_DLL',
 'CHOCO_CMD',
 'MIPCL_CMD',
 'SCIP_CMD']

In [28]:
solver = pulp.PULP_CBC_CMD(msg=False, threads=4)
Prob.solve(solver)

"""Final Result"""
print("Opt Result \n")
for v in Prob.variables():
    print(f"{v.getName()}={v.value()}")
    
"""Constraints values"""
print("\n\n Constraints \n")
for name, c in Prob.constraints.items():
    value = c.value()-c.constant
    print(c)
    print(f"{name} = {value}\n")
    

Opt Result 

Beef=60.0
Chicken=0.0
Gel=40.0
Mutton=0.0
Rice=0.0
Wheat=0.0


 Constraints 

Beef + Chicken + Gel + Mutton + Rice + Wheat = 100
Percentage = 100.0

0.2*Beef + 0.1*Chicken + 0.15*Mutton + 0.04*Wheat >= 8.0
Protein = 12.0

0.1*Beef + 0.08*Chicken + 0.11*Mutton + 0.01*Rice + 0.01*Wheat >= 6.0
Fat = 6.0

0.005*Beef + 0.001*Chicken + 0.003*Mutton + 0.1*Rice + 0.15*Wheat <= 2.0
Fibre = 0.30000000000000004

0.005*Beef + 0.002*Chicken + 0.007*Mutton + 0.002*Rice + 0.008*Wheat <= 0.4
Salt = 0.3

